In [ ]:
import pandas as pd
import re
import numpy as np


In [238]:
ruta = 'comentarios_Yamandú Orsi_procesado_Ortografiado_sentimiento_analizado'
df = pd.read_csv(f'{ruta}.csv')

In [239]:
df.sample(2
        )

,Comentario,Autor,Likes_Comentario,Fecha_Publicacion_Comentario,URL_Video,Titulo_Video,Descripcion_Video,Tags,Fecha_Publicacion_Video,Idioma,...,Comentario_nrc,Titulo_Video_afinn,Titulo_Video_nrc,Descripcion_Video_afinn,Descripcion_Video_nrc,Comentario_Polaridad,Comentario_Subjetividad,Categoria_Nombre,Tags_afinn,Tags_nrc
6283,Blablablá gobernar forma transversal,@crisgallinal7544,0,2024-03-11T00:03:59Z,https://www.youtube.com/watch?v=vylJ7yLP4aY,cosse si gana yamandú segunda fa define gusto ...,candidata frenteamplista lanzó semana forma of...,desayunos informales canal magazine periodísti...,2024-03-06T12:41:31Z,NaN,...,"positivo:2, confianza:1",2,"alegría:1, positivo:1, confianza:1",-4,"positivo:9, confianza:7, anticipación:4, negat...",0.000000,0.000000,Entertainment,0,"asco:1, anticipación:1, alegría:1, positivo:1,..."
6219,Acá pretende denuncia justamente aplicar ley v...,@wilsonperalta844,0,2024-03-13T20:07:43Z,https://www.youtube.com/watch?v=bDjU9i189xA,mujer trans presentó denuncia seccional yamand...,acusó violencia física tras episodio sucedido,boca debate actualidad política sociedad victo...,2024-03-13T18:31:32Z,NaN,...,"enfado:5, asco:5, miedo:3, negativo:9, confian...",-2,"enfado:1, asco:1, miedo:1, negativo:1",-3,"enfado:1, miedo:1, negativo:1, tristeza:1, pos...",-0.071429,0.571429,People & Blogs,0,"sorpresa:1, positivo:3, confianza:3, enfado:1,..."


# Rescatar Tags, Paises y Emociones

In [240]:
from collections import Counter
# Inicializa un contador para sumar las emociones
contador_emociones = Counter()

# Recorremos cada fila de la columna
for fila in df['Comentario_nrc'].dropna():
    emociones = fila.split(',')  # Separar por coma
    for emocion in emociones:
        emocion = emocion.strip()
        if ':' in emocion:
            clave, valor = emocion.split(':')
            clave = clave.strip().lower()
            try:
                valor = int(valor.strip())
                contador_emociones[clave] += valor
            except ValueError:
                continue  # Ignora si no puede convertir el valor

# Convertimos el contador a DataFrame ordenado
resumen_df = pd.DataFrame(contador_emociones.items(), columns=['Emocion', 'Total'])
resumen_df = resumen_df.sort_values(by='Total', ascending=False)

# Guardar el resumen como CSV
resumen_df.to_csv('resumen_emociones_nrc.csv', index=False, encoding='utf-8-sig')

print("✅ Resumen guardado en 'resumen_emociones_nrc.csv'")

✅ Resumen guardado en 'resumen_emociones_nrc.csv'


In [241]:
from collections import Counter
# 2. Procesar columna Tags y contar los más frecuentes
def contar_tags(df, columna='Tags'):
    todos_los_tags = []

    for tags in df[columna].dropna():
        # Separar tags por espacio
        separados = tags.split()
        todos_los_tags.extend(separados)

    # Contar frecuencia con Counter
    conteo = Counter(todos_los_tags)
    df_resultado = pd.DataFrame(conteo.items(), columns=['Tag', 'Frecuencia'])
    df_resultado = df_resultado.sort_values(by='Frecuencia', ascending=False).reset_index(drop=True)
    return df_resultado


In [242]:
print(df['Pais_Canal'].unique())
conteo_paises = df['Pais_Canal'].value_counts().reset_index()
conteo_paises.columns = ['País', 'Frecuencia']  # Renombrar columnas
conteo_paises['Porcentaje'] = (conteo_paises['Frecuencia'] / len(df)) * 100  # Añadir porcentaje

print(conteo_paises)

['UY' nan 'ES' 'PE' 'AR' 'FR' 'IN' 'US']
  País  Frecuencia  Porcentaje
0   UY       25762   95.006638
1   ES         334    1.231745
2   AR         104    0.383537
3   FR          46    0.169642
4   US           7    0.025815
5   PE           5    0.018439
6   IN           3    0.011064


In [243]:
df_tags = contar_tags(df)
print("\n🎯 Tags más frecuentes:\n")
print(df_tags.head(20))  
# Guardar como CSV en la misma carpeta del notebook
df_tags.to_csv("tags_contados.csv", index=False)
print("✅ Archivo CSV guardado como: tags_contados.csv")



🎯 Tags más frecuentes:

            Tag  Frecuencia
0       uruguay       24221
1          orsi       22836
2       yamandú       21023
3        frente       10105
4        amplio       10090
5    elecciones        9486
6    montevideo        5214
7    presidente        5201
8         canal        4201
9     canelones        4146
10    encuestas        4044
11       jingle        4010
12   actualidad        3952
13     victoria        3072
14     política        2826
15  entrevistas        2536
16      familia        2533
17        cosse        2496
18     carolina        2495
19   observador        2117
✅ Archivo CSV guardado como: tags_contados.csv


## Viztazo a los Datos: reporte_datos.txt


In [244]:
def analizar_comentarios(df, log):
    log.append("\n🔍 Tipo real de objetos en la columna 'Comentario':")
    tipos = df['Comentario'].map(type).value_counts()
    log.append(str(tipos))

    log.append("\n⚠️ Ejemplos de valores no str (si existen):")
    no_str = df[~df['Comentario'].map(lambda x: isinstance(x, str))]
    log.append(str(no_str.head()))

    

def analizar_calidad_datos(df, log):
    log.append("\n📊 INFORME GENERAL DEL DATAFRAME")
    log.append(f"Filas: {df.shape[0]}, Columnas: {df.shape[1]}")

    log.append("\n🧠 Tipos de datos por columna:")
    log.append(str(df.dtypes))

    log.append("\n🔎 Valores únicos por columna:")
    log.append(str(df.nunique()))

    log.append("\n🚫 Conteo de valores nulos (NaN) por columna:")
    log.append(str(df.isna().sum()))

    log.append("\n📉 Porcentaje de valores nulos por columna:")
    log.append(str((df.isna().mean() * 100).round(2).astype(str) + '%'))


def guardar_log(log):
    with open("reporte_datos.txt", "w", encoding="utf-8") as f:
        for linea in log:
            f.write(str(linea) + "\n")
    print("📄 Informe guardado como 'reporte_datos.txt'")

In [245]:
log = []
analizar_calidad_datos(df, log)
if 'Comentario' in df.columns:
    analizar_comentarios(df, log)
else:
    log.append("\n❌ La columna 'Comentario' no está en el archivo.")

guardar_log(log)

📄 Informe guardado como 'reporte_datos.txt'


## PreProcesamiento de Datos para Machine Learning: ML

| **Columna**                    | **¿Usable?** | **Cómo procesarla / convertirla**                                 |
| ------------------------------ | ------------ | ----------------------------------------------------------------- |
| `Comentario`                   | ✅            | NLP: limpiar, tokenizar, vectorizar (TF-IDF, embeddings)          |
| `Autor`                        | 🔸           | Poco útil directamente; posible usar como frecuencia si se repite |
| `Likes_Comentario`             | ✅            | Escalar (MinMaxScaler o StandardScaler)                           |
| `Fecha_Publicacion_Comentario` | ✅            | Extraer año, mes, día, hora como nuevas columnas                  |
| `URL_Video`                    | ❌            | No útil para ML; solo identificador                               |
| `Titulo_Video`                 | ✅            | NLP: igual que `Comentario`                                       |
| `Descripcion_Video`            | ✅            | NLP: igual que `Comentario`                                       |
| `Tags`                         | ✅            | Tokenizar, contar palabras clave, binarizar o vectorizar          |
| `Fecha_Publicacion_Video`      | ✅            | Extraer año, mes, hora para análisis temporal                     |
| `Idioma`                       | ✅            | Codificar como categórica (One-Hot o LabelEncoding)               |
| `Duracion`                     | ✅            | Convertir de ISO 8601 (`PT2H50M25S`) a minutos totales            |
| `Numero_Views`                 | ✅            | Escalar                                                           |
| `Numero_Likes`                 | ✅            | Escalar                                                           |
| `Numero_Comentarios`           | ✅            | Escalar                                                           |
| `Pais_Canal`                   | ✅            | Codificar país (categórica)                                       |
| `Subscriptores_Canal`          | ✅            | Escalar                                                           |
| `Canal_Titulo`                 | 🔸           | Útil si se agrupa o analiza como canal dominante                  |
| `Comentario_afinn`             | ✅            | Escalar (sentimiento ya numerizado)                               |
| `Comentario_nrc`               | ✅            | Contar emociones y expandir como columnas: alegría, miedo, etc.   |
| `Titulo_Video_afinn`           | ✅            | Escalar                                                           |
| `Titulo_Video_nrc`             | ✅            | Igual que `Comentario_nrc`                                        |
| `Descripcion_Video_afinn`      | ✅            | Escalar                                                           |
| `Descripcion_Video_nrc`        | ✅            | Igual que `Comentario_nrc`                                        |
| `Comentario_Polaridad`         | ✅            | Escalar (valor entre -1 y 1)                                      |
| `Comentario_Subjetividad`      | ✅            | Escalar (valor entre 0 y 1)                                       |
| `Tags_afinn`                   | ✅            | Escalar                                                           |
| `Tags_nrc`                     | ✅            | Expandir emociones como columnas                                  |
| `Categoria_Nombre`             | ✅            | Codificar como variable categórica                                |


In [ ]:

# --- Eliminar columnas específicas ---
def eliminar_columnas(df, columnas_a_eliminar, log):
    columnas_eliminadas = [col for col in columnas_a_eliminar if col in df.columns]
    df = df.drop(columns=columnas_eliminadas, errors='ignore')
    log.append(f"🗑️ Columnas eliminadas: {columnas_eliminadas}")
    return df
def ponderar_pais(df, columna):
    total_paises = df[columna].dropna().value_counts(normalize=True)
    ponderacion = df[columna].map(total_paises).fillna(0)
    return ponderacion
def parsear_duracion_iso8601(duracion):
    if not isinstance(duracion, str) or not duracion.startswith('PT'):
        return 0
    horas = minutos = segundos = 0

    match_h = re.search(r'(\d+)H', duracion)
    match_m = re.search(r'(\d+)M', duracion)
    match_s = re.search(r'(\d+)S', duracion)

    if match_h:
        horas = int(match_h.group(1))
    if match_m:
        minutos = int(match_m.group(1))
    if match_s:
        segundos = int(match_s.group(1))

    total_segundos = horas * 3600 + minutos * 60 + segundos
    return total_segundos
def ponderar_categoria(df):
    # Diccionario de pesos por categoría PONDERACION OBTENIDA GRACIAS A CHATGPT 4o
    pesos_categoria = {
        "News & Politics": 1.0,
        "Education": 0.8,
        "People & Blogs": 0.7,
        "Entertainment": 0.6,
        "Comedy": 0.6,
        "Music": 0.5,
        "Howto & Style": 0.5,
        "Science & Technology": 0.7,
        "Sports": 0.4,
        "Gaming": 0.3,
        "Film & Animation": 0.4,
        "Autos & Vehicles": 0.2,
        "Pets & Animals": 0.2,
        "Travel & Events": 0.3,
        "Videoblogging": 0.5,
        "Short Movies": 0.4,
        "Movies": 0.3,
        "Nonprofits & Activism": 0.6,
        "Anime/Animation": 0.3,
        "Action/Adventure": 0.3,
        "Classics": 0.3,
        "Comedy (Movies)": 0.5,
        "Documentary": 0.7,
        "Drama": 0.4,
        "Family": 0.4,
        "Foreign": 0.3,
        "Horror": 0.3,
        "Sci-Fi/Fantasy": 0.3,
        "Thriller": 0.3,
        "Shorts": 0.4,
        "Shows": 0.4,
        "Trailers": 0.2
    }

    # Asigna el peso según la categoría (NaN si no está en el diccionario)
    df['Categoria_Ponderada'] = df['Categoria_Nombre'].map(pesos_categoria)

    # Rellena valores faltantes con un peso neutro o bajo, por ejemplo 0.3
    df['Categoria_Ponderada'] = df['Categoria_Ponderada'].fillna(0.3)

    # Elimina la columna original
    df.drop(columns=['Categoria_Nombre'], inplace=True)

    return df

# --- Rellenar valores nulos ---
def rellenar_nulos(df, log):
    log.append("\n🧽 Relleno de valores nulos:")
    for col in df.columns:
        if df[col].isna().sum() > 0:
            if df[col].dtype == 'object':
                if df[col].nunique() < 30:
                    modo = df[col].mode(dropna=True)
                    valor = modo[0] if not modo.empty else "desconocido"
                    df[col] = df[col].fillna(valor)
                    log.append(f"  - {col}: categórica → rellenado con '{valor}'")
                else:
                    df[col] = df[col].fillna("")
                    log.append(f"  - {col}: texto → rellenado con cadena vacía")
            elif np.issubdtype(df[col].dtype, np.number):
                mediana = df[col].median()
                df[col] = df[col].fillna(mediana)
                log.append(f"  - {col}: numérica → rellenado con mediana ({mediana})")
    return df

# --- Expandir columnas _nrc a columnas de emociones individuales ---
def expandir_emociones_nrc(df, log):
    columnas_nrc = [col for col in df.columns if col.endswith('_nrc')]
    emociones_total = set()

    log.append("\n🌈 Expansión de columnas _nrc:")
    for col in columnas_nrc:
        log.append(f"  - Procesando columna: {col}")
        emociones_dicts = df[col].fillna("").apply(lambda x: {
            k.strip(): int(v) for k, v in (item.split(":") for item in x.split(",") if ":" in item)
        })

        emociones_col = set()
        for dic in emociones_dicts:
            emociones_col.update(dic.keys())
        emociones_total.update(emociones_col)

        for emocion in emociones_col:
            df[f"{col}_{emocion}"] = emociones_dicts.apply(lambda d: d.get(emocion, 0))

        df.drop(columns=[col], inplace=True)

    log.append(f"  - Emociones detectadas: {sorted(list(emociones_total))}")
    return df

# --- Guardar resumen del procesamiento ---
def guardar_resumen(log, ruta_salida_base):
    resumen_path = f"{ruta_salida_base}_resumen_procesamiento.txt"
    with open(resumen_path, "w", encoding="utf-8") as f:
        for linea in log:
            f.write(linea + "\n")
    print(f"📄 Resumen guardado como: {resumen_path}")

### Manejo de fechas

In [ ]:

# Asegúrate de que la columna sea tipo datetime
df['Fecha_Publicacion_Comentario'] = pd.to_datetime(df['Fecha_Publicacion_Comentario'], errors='coerce')

# 1. Extraer componentes temporales (útiles para estacionalidad y análisis temporal)
df['comentario_anio'] = df['Fecha_Publicacion_Comentario'].dt.year
df['comentario_mes'] = df['Fecha_Publicacion_Comentario'].dt.month
df['comentario_dia'] = df['Fecha_Publicacion_Comentario'].dt.day
df['comentario_dia_semana'] = df['Fecha_Publicacion_Comentario'].dt.weekday  # Lunes = 0
df['comentario_hora'] = df['Fecha_Publicacion_Comentario'].dt.hour

# 2. Convertir a timestamp numérico (útil para modelos que capturan tendencia temporal)
# Esto convierte la fecha a segundos desde 1970-01-01 (Unix Epoch)
df['comentario_timestamp'] = df['Fecha_Publicacion_Comentario'].astype('int64') // 10**9

# (Opcional) Revisar si alguna fila no pudo convertirse correctamente
errores_fecha = df['Fecha_Publicacion_Comentario'].isnull().sum()
if errores_fecha > 0:
    print(f"⚠️ Atención: {errores_fecha} valores no pudieron convertirse a datetime.")


In [248]:
##PARA REDES NEURONALES O REGRESION LINEAL
'''from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
df['comentario_timestamp_norm'] = scaler.fit_transform(df[['comentario_timestamp']])'''


"from sklearn.preprocessing import MinMaxScaler\nscaler = MinMaxScaler()\ndf['comentario_timestamp_norm'] = scaler.fit_transform(df[['comentario_timestamp']])"

In [249]:
# --- BLOQUE DE EJECUCIÓN PARA NOTEBOOK ---

log = [f"📂 Procesando DataFrame desde ruta: {ruta}.csv"]

# Paso 1: Eliminar columnas no deseadas
columnas_a_eliminar = [
    'Comentario', 'Titulo_Video', 'Descripcion_Video',
    'Fecha_Publicacion_Video', 'Idioma','Autor','URL_Video','Tags','Canal_Titulo','Fecha_Publicacion_Comentario'
]
df = eliminar_columnas(df, columnas_a_eliminar, log)

# Paso 2: Rellenar valores nulos
df = rellenar_nulos(df, log)
# Ponderar países
df['Ponderacion_Pais_Canal'] = ponderar_pais(df, 'Pais_Canal')
df = df.drop(columns=['Pais_Canal'], errors='ignore')
# Paso 3: Expandir emociones NRC
df = expandir_emociones_nrc(df, log)
df= ponderar_categoria(df)
# Parsear Duracion
df['Duracion_Segundos'] = df['Duracion'].apply(parsear_duracion_iso8601)
df = df.drop(columns=['Duracion'])  # opcional: elimina columna original
# Paso 4: Guardar CSV y resumen
ruta_salida_base = ruta + "_ML"
df.to_csv(f"{ruta_salida_base}.csv", index=False, encoding='utf-8-sig')
log.append(f"\n💾 Archivo final guardado como: {ruta_salida_base}.csv")

guardar_resumen(log, ruta_salida_base)

print("✅ Procesamiento finalizado.")


📄 Resumen guardado como: comentarios_Yamandú Orsi_procesado_Ortografiado_sentimiento_analizado_ML_resumen_procesamiento.txt
✅ Procesamiento finalizado.


# Features Derivados recomendados

Ponderación del Comentario

Ponderador = (Likes_Comentario + 1) / (Numero_Comentarios + 1)
→ Más peso a comentarios más relevantes dentro de su contexto.

Engagement del Video

Engagement = (Numero_Likes + Numero_Comentarios) / (Numero_Views + 1)
→ Refleja cuánto interactúa el público con ese video.

Influencia del Canal

Influencia_Canal = log(Subscriptores_Canal + 1)
→ Escala logarítmica para evitar distorsión por canales gigantes.

Polaridad ponderada

Comentario_Polaridad_Ponderada = Comentario_Polaridad * Ponderador

Subjetividad ponderada

Comentario_Subjetividad_Ponderada = Comentario_Subjetividad * Ponderador

AFINN ponderado

Comentario_afinn_Ponderado = Comentario_afinn * Ponderador

Emoción codificada (One-hot o embedding)
→ Convertir Comentario_nrc_emocion a variables binarias como:

emocion_positivo, emocion_miedo, emocion_confianza, etc.

In [ ]:


def crear_features_avanzados(df):
    # Prevenir división por cero
    df['Likes_Comentario'] = df['Likes_Comentario'].fillna(0)
    df['Numero_Comentarios'] = df['Numero_Comentarios'].replace(0, np.nan)
    df['Numero_Views'] = df['Numero_Views'].replace(0, np.nan)
    df['Numero_Likes'] = df['Numero_Likes'].replace(0, np.nan)
    df['Subscriptores_Canal'] = df['Subscriptores_Canal'].replace(0, np.nan)

    # --- 1. Ponderador del Comentario ---
    df['Ponderador_Comentario'] = (df['Likes_Comentario'] + 1) / (df['Numero_Comentarios'] + 1)

    # --- 2. Engagement del Video ---
    df['Engagement_Video'] = (df['Numero_Likes'] + df['Numero_Comentarios']) / (df['Numero_Views'] + 1)

    # --- 3. Influencia del Canal ---
    df['Influencia_Canal'] = np.log1p(df['Subscriptores_Canal'])  # log(Subscriptores_Canal + 1)

    # --- 4. Polaridad y Subjetividad Ponderadas ---
    df['Comentario_Polaridad_Ponderada'] = df['Comentario_Polaridad'] * df['Ponderador_Comentario']
    df['Comentario_Subjetividad_Ponderada'] = df['Comentario_Subjetividad'] * df['Ponderador_Comentario']

    # --- 5. AFINN Ponderado ---
    df['Comentario_afinn_Ponderado'] = df['Comentario_afinn'] * df['Ponderador_Comentario']
    df['Titulo_Video_afinn_Ponderado'] = df['Titulo_Video_afinn'] * df['Engagement_Video']
    df['Descripcion_Video_afinn_Ponderado'] = df['Descripcion_Video_afinn'] * df['Engagement_Video']
    df['Tags_afinn_Ponderado'] = df['Tags_afinn'] * df['Influencia_Canal']

    # --- 6. Emociones NRC Ponderadas ---
    emociones = [
        'alegría', 'anticipación', 'enfado', 'tristeza', 'positivo', 'sorpresa',
        'asco', 'negativo', 'confianza', 'miedo'
    ]
    fuentes = ['Comentario_', 'Titulo_Video_', 'Descripcion_Video_', 'Tags_']

    for emocion in emociones:
        for fuente in fuentes:
            col = f'{fuente}nrc_{emocion}'
            if col in df.columns:
                nueva_col = f'{col}_Ponderada'
                if fuente == 'Comentario_':
                    df[nueva_col] = df[col] * df['Ponderador_Comentario']
                elif fuente == 'Titulo_Video_' or fuente == 'Descripcion_Video_':
                    df[nueva_col] = df[col] * df['Engagement_Video']
                else:
                    df[nueva_col] = df[col] * df['Influencia_Canal']

    # --- 7. Rellenar nulos generados ---
    df.fillna(0, inplace=True)

    return df


In [251]:
df_features= crear_features_avanzados(df)
df_features.sample(10)

C:\Users\HP\AppData\Local\Temp\ipykernel_2836\3546617671.py:46: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nueva_col] = df[col] * df['Engagement_Video']
C:\Users\HP\AppData\Local\Temp\ipykernel_2836\3546617671.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[nueva_col] = df[col] * df['Influencia_Canal']
C:\Users\HP\AppData\Local\Temp\ipykernel_2836\3546617671.py:44: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consi

,Likes_Comentario,Numero_Views,Numero_Likes,Numero_Comentarios,Subscriptores_Canal,Comentario_afinn,Titulo_Video_afinn,Descripcion_Video_afinn,Comentario_Polaridad,Comentario_Subjetividad,...,Descripcion_Video_nrc_negativo_Ponderada,Tags_nrc_negativo_Ponderada,Comentario_nrc_confianza_Ponderada,Titulo_Video_nrc_confianza_Ponderada,Descripcion_Video_nrc_confianza_Ponderada,Tags_nrc_confianza_Ponderada,Comentario_nrc_miedo_Ponderada,Titulo_Video_nrc_miedo_Ponderada,Descripcion_Video_nrc_miedo_Ponderada,Tags_nrc_miedo_Ponderada
3565,7,66384,2308.0,1106,140000,1,0,0,0.000000,0.000000,...,0.000000,0.0,0.007227,0.051427,0.000000,0.000000,0.000000,0.102855,0.000000,0.0
23928,1,131771,2500.0,2042,36600,-2,0,0,-0.800000,0.200000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000979,0.000000,0.000000,0.0
15672,1,21235,238.0,268,85000,2,0,1,0.172619,0.357143,...,0.000000,0.0,0.014870,0.047655,0.119137,0.000000,0.000000,0.000000,0.023827,0.0
6243,0,19017,192.0,426,52600,-6,-2,-3,0.000000,0.000000,...,0.032496,0.0,0.004684,0.000000,0.000000,32.611471,0.004684,0.032496,0.032496,0.0
17895,15,128791,4267.0,883,140000,5,0,0,0.525000,0.750000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
14498,0,46117,2661.0,383,140000,0,-1,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
15245,20,5705,255.0,75,210000,5,0,0,0.262500,0.525000,...,0.000000,0.0,0.552632,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
12482,4,11666,299.0,229,30100,0,0,0,0.000000,0.000000,...,0.000000,0.0,0.021739,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
27041,4,3981,310.0,28,210000,1,2,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
17602,3,128791,4267.0,883,140000,0,0,0,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0


In [252]:
ruta_salida_fealtures = ruta_salida_base + "_fealtures"
df_features.to_csv(f"{ruta_salida_fealtures}.csv", index=False, encoding='utf-8-sig')
print(f"\n💾 Archivo final guardado como: {ruta_salida_fealtures}.csv")


💾 Archivo final guardado como: comentarios_Yamandú Orsi_procesado_Ortografiado_sentimiento_analizado_ML_fealtures.csv


In [253]:

contador = 0
for columna in df.columns:
    if contador >= 5:  # Detener después de 5 columnas
        break
        
    print(f"Columna: {columna}")
    print(df[columna].unique())  # Valores únicos
    print("-" * 50)  # Separador visual
    contador += 1  # Incrementar el contador

print("\nSe han mostrado las primeras 5 columnas del DataFrame.")

Columna: Likes_Comentario
[  0   1   3   5   4   2   8   6  11 178  16  21  31  37  20  23  17  30
   7  12  18  65  13  10  19  55   9 132  14 107 242 162 104  82  72  24
  45 106  48  35  77  58  47 135 259  28  26  44  94 122  42  39  25  86
  38  68  57  49 127  43  29  15  22  27  36  34  40  62  59  90  88  41
  84  92  78  53  32  66  73  33  54  70 137  64 192 103 223  99 158  81
 119 147 163 203  89 139  98  56  61  80  46  96 173 182 306  93 290 116
 288 111  67  85 133 129  51 180  79  52  50  71 143 165 124 161 121 131
 115 113 254 175  95 154  91 394 176 195 118 160 200  87  83  63 142  97
 151 170 168 194 109  69 123  60 117  75 181  76 202 110 240 201 228 234
 185 105 108 140 102 128 153 341 398 458 340 136 150 221 114 196 130 145
 204 167 251 146 305 213 439 100 141 217 174 336  74 169 400 101 172 233
 317 227 193 346 313 414 183 236 261 120]
--------------------------------------------------
Columna: Numero_Views
[  73724   48276   23522   22313   16364   15905   15645